In [149]:
%load_ext autoreload
%autoreload 2

import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import openml
from sklearn import preprocessing

from utils import preprocess_datasets, fit_one_hot_encoder, encode_with_one_hot_encoder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
openml_list = openml.datasets.list_datasets([6,6332])
data_list = pd.DataFrame.from_dict(openml_list, orient='index')

/tmp/ipykernel_110980/2702947544.py:1: FutureWarning: Support for `output_format` of 'dict' will be removed in 0.15 and pandas dataframes will be returned instead. To ensure your code will continue to work, use `output_format`='dataframe'.
  openml_list = openml.datasets.list_datasets([6,6332])


In [101]:
data_list

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
6,6,letter,1,1,active,ARFF,813.0,26.0,734.0,26.0,17.0,20000.0,0.0,0.0,16.0,1.0
6332,6332,cylinder-bands,2,2,active,ARFF,312.0,71.0,228.0,2.0,40.0,540.0,263.0,999.0,18.0,22.0


In [102]:
entry = data_list.loc[6332]
dataset = openml.datasets.get_dataset(int(entry.did))

/home/wei/anaconda3/lib/python3.11/site-packages/openml/datasets/functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [103]:
dataset.features

{0: [0 - timestamp (nominal)],
 1: [1 - cylinder_number (nominal)],
 2: [2 - customer (nominal)],
 3: [3 - job_number (numeric)],
 4: [4 - grain_screened (nominal)],
 5: [5 - ink_color (nominal)],
 6: [6 - proof_on_ctd_ink (nominal)],
 7: [7 - blade_mfg (nominal)],
 8: [8 - cylinder_division (nominal)],
 9: [9 - paper_type (nominal)],
 10: [10 - ink_type (nominal)],
 11: [11 - direct_steam (nominal)],
 12: [12 - solvent_type (nominal)],
 13: [13 - type_on_cylinder (nominal)],
 14: [14 - press_type (nominal)],
 15: [15 - press (nominal)],
 16: [16 - unit_number (numeric)],
 17: [17 - cylinder_size (nominal)],
 18: [18 - paper_mill_location (nominal)],
 19: [19 - plating_tank (nominal)],
 20: [20 - proof_cut (numeric)],
 21: [21 - viscosity (numeric)],
 22: [22 - caliper (nominal)],
 23: [23 - ink_temperature (numeric)],
 24: [24 - humifity (numeric)],
 25: [25 - roughness (numeric)],
 26: [26 - blade_pressure (numeric)],
 27: [27 - varnish_pct (numeric)],
 28: [28 - press_speed (numeric

In [104]:
dataset.default_target_attribute

'band_type'

In [227]:
x, y, cat_indicators, attr_names = dataset.get_data(dataset_format='dataframe', target=dataset.default_target_attribute)

In [228]:
x

,customer,job_number,grain_screened,ink_color,proof_on_ctd_ink,blade_mfg,cylinder_division,paper_type,ink_type,direct_steam,...,ink_pct,solvent_pct,ESA_Voltage,ESA_Amperage,wax,hardener,roller_durometer,current_density,anode_space_ratio,chrome_content
0,tvguide,25503,yes,key,yes,benton,gallatin,uncoated,uncoated,no,...,50.5,36.4,0.0,0.0,2.5,1.0,34.0,40,105.00,100
1,tvguide,25503,yes,key,yes,benton,gallatin,uncoated,uncoated,no,...,54.9,38.5,0.0,0.0,2.5,0.7,34.0,40,105.00,100
2,modmat,47201,yes,key,yes,benton,gallatin,uncoated,coated,no,...,53.8,39.8,0.0,0.0,2.8,0.9,40.0,40,103.87,100
3,massey,39039,yes,key,yes,benton,gallatin,uncoated,uncoated,no,...,55.6,38.8,0.0,0.0,2.5,1.3,40.0,40,108.06,100
4,kmart,37351,no,key,yes,benton,gallatin,uncoated,coated,no,...,57.5,42.5,5.0,0.0,2.3,0.6,35.0,40,106.67,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,kmart,85813,NaN,key,NaN,NaN,gallatin,super,uncoated,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,40,112.50,100
536,best,38064,NaN,key,NaN,NaN,gallatin,super,coated,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,40,110.00,100
537,kmart,85814,NaN,key,NaN,NaN,gallatin,super,coated,NaN,...,NaN,NaN,NaN,NaN,2.7,2.8,NaN,40,108.00,100
538,kmart,85814,NaN,key,NaN,NaN,gallatin,super,uncoated,NaN,...,NaN,NaN,NaN,NaN,1.5,2.3,NaN,40,108.00,100


In [229]:
preprocess_datasets(x,x.copy(),x.copy())

Data preprocess finished! Dropped 2 features


In [230]:
x

,customer,job_number,grain_screened,proof_on_ctd_ink,blade_mfg,paper_type,ink_type,direct_steam,solvent_type,type_on_cylinder,...,ink_pct,solvent_pct,ESA_Voltage,ESA_Amperage,wax,hardener,roller_durometer,current_density,anode_space_ratio,chrome_content
0,tvguide,-1.351296,yes,yes,benton,uncoated,uncoated,no,line,yes,...,-9.769617e-01,-6.545847e-01,-5.670761e-01,-9.697692e-02,0.185585,0.035459,-1.817417e-01,40,0.393220,100
1,tvguide,-1.351296,yes,yes,benton,uncoated,uncoated,no,line,yes,...,-1.404133e-01,-2.042730e-02,-5.670761e-01,-9.697692e-02,0.185585,-0.786264,-1.817417e-01,40,0.393220,100
2,modmat,1.136747,yes,yes,benton,uncoated,coated,no,line,yes,...,-3.495504e-01,3.721463e-01,-5.670761e-01,-9.697692e-02,0.738199,-0.238449,1.224823e+00,40,0.166033,100
3,massey,0.200835,yes,yes,benton,uncoated,uncoated,no,line,yes,...,-7.326083e-03,7.016661e-02,-5.670761e-01,-9.697692e-02,0.185585,0.857182,1.224823e+00,40,1.008434,100
4,kmart,0.007278,no,yes,benton,uncoated,coated,no,line,yes,...,3.539107e-01,1.187492e+00,1.581984e+00,-9.697692e-02,-0.182825,-1.060171,5.268575e-02,40,0.728974,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,kmart,5.564265,no,yes,benton,super,uncoated,no,line,yes,...,2.701834e-15,-2.145695e-15,-9.543746e-17,1.764122e-17,-2.577486,0.035459,-1.665707e-15,40,1.901098,100
536,best,0.089035,no,yes,benton,super,coated,no,line,yes,...,2.701834e-15,-2.145695e-15,-9.543746e-17,1.764122e-17,-4.419534,-2.703617,-1.665707e-15,40,1.398472,100
537,kmart,5.564380,no,yes,benton,super,coated,no,line,yes,...,2.701834e-15,-2.145695e-15,-9.543746e-17,1.764122e-17,0.553994,4.965795,-1.665707e-15,40,0.996371,100
538,kmart,5.564380,no,yes,benton,super,uncoated,no,line,yes,...,2.701834e-15,-2.145695e-15,-9.543746e-17,1.764122e-17,-1.656463,3.596257,-1.665707e-15,40,0.996371,100


In [232]:
one_hot_encoder_raw = preprocessing.OneHotEncoder(handle_unknown='ignore', drop='if_binary', sparse=False)
one_hot_encoder = fit_one_hot_encoder(one_hot_encoder_raw, x)

/home/wei/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [258]:
x.iloc[0]

customer                tvguide
job_number            -1.351296
grain_screened              yes
proof_on_ctd_ink            yes
blade_mfg                benton
paper_type             uncoated
ink_type               uncoated
direct_steam                 no
solvent_type               line
type_on_cylinder            yes
press_type             motter94
press                       821
unit_number           -0.886077
cylinder_size           tabloid
paper_mill_location     northus
plating_tank               1911
proof_cut              1.163695
viscosity             -0.616479
caliper                     0.2
ink_temperature         1.28384
humifity              -0.071166
roughness              0.135805
blade_pressure        -1.273526
varnish_pct            1.130573
press_speed           -0.379764
ink_pct               -0.976962
solvent_pct           -0.654585
ESA_Voltage           -0.567076
ESA_Amperage          -0.096977
wax                    0.185585
hardener               0.035459
roller_d

In [233]:
x1=one_hot_encoder.transform(x)

In [261]:
x1

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         3.54589513e-02, -1.81741680e-01,  3.93219551e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -7.86263703e-01, -1.81741680e-01,  3.93219551e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.38448600e-01,  1.22482292e+00,  1.66032571e-01],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.96579487e+00, -1.66570709e-15,  9.96370826e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         3.59625712e+00, -1.66570709e-15,  9.96370826e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         3.54589513e-02, -1.66570709e-15,  1.01647587e+00]])